<a href="https://colab.research.google.com/github/saharshhhh/Generative_AI/blob/main/HF_BERT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --quiet

In [ ]:
!pip install -q huggingface_hub

In [ ]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True, output_attentions = True)
inputs=tokenizer("Hello, how are you?", return_tensors="pt")
outputs=model(**inputs)

last_hidden_states=outputs.last_hidden_state
pooler_output=outputs.pooler_output
hidden_states=outputs.hidden_states
attentions=outputs.attentions

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
len(attentions)

12

In [ ]:
print("Shape of last_hidden_states:", last_hidden_states.shape)
print("Shape of pooler_output:", pooler_output.shape)

print("Shape of attentions (tuple of tensors, each tensor shape):")
for i, attention in enumerate(attentions):
    print(f"  Layer {i}: {attention.shape}")

Shape of last_hidden_states: torch.Size([1, 8, 768])
Shape of pooler_output: torch.Size([1, 768])
Shape of attentions (tuple of tensors, each tensor shape):
  Layer 0: torch.Size([1, 12, 8, 8])
  Layer 1: torch.Size([1, 12, 8, 8])
  Layer 2: torch.Size([1, 12, 8, 8])
  Layer 3: torch.Size([1, 12, 8, 8])
  Layer 4: torch.Size([1, 12, 8, 8])
  Layer 5: torch.Size([1, 12, 8, 8])
  Layer 6: torch.Size([1, 12, 8, 8])
  Layer 7: torch.Size([1, 12, 8, 8])
  Layer 8: torch.Size([1, 12, 8, 8])
  Layer 9: torch.Size([1, 12, 8, 8])
  Layer 10: torch.Size([1, 12, 8, 8])
  Layer 11: torch.Size([1, 12, 8, 8])


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name="textattack/bert-base-uncased-SST-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
text= "I absolutely dont like the movie. it was worst"

inputs = tokenizer(text, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 1045, 7078, 2123, 2102, 2066, 1996, 3185, 1012, 2009, 2001, 5409,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

PyTorch

In [ ]:
with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits
  probs= F.softmax(logits, dim=1)
  predicted_class = torch.argmax(probs, dim=1).item()

Tensorflow

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

class BertClassifier(tf.keras.Model):
  def __init__(self, model_name='bert-base-uncased', nums_classes=2):
    super(BertClassifier, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name)
    self.dropout = tf.keras.layers.Dropout(0.3)
    self.classifier = tf.keras.layers.Dense(nums_classes, activation='softmax')

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
    outputs = self.bert(
        inputs,
        attention_mask=inputs['attention_mask'],
        token_type_ids=inputs.get('token_type_ids', None),
        training=training
    )
    cls_output = outputs.pooler_output
    x = self.dropout(cls_output, training=training)
    x = self.classifier(x)
    return x

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "I really loved the new iPhone!"

encoded_input = tokenizer(text, return_tensors='tf', padding=True, truncation=True)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [ ]:
model = BertClassifier(nums_classes=2)

model.trainable = False
logits = model(encoded_input, training=False)
predicted_class = tf.argmax(logits, axis=1).numpy()[0]
confidence = tf.reduce_max(logits, axis=1).numpy()[0]

print(f"Predicted class: {predicted_class}, Confidence: {confidence:.4f}")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were 

Predicted class: 0, Confidence: 0.6669
